# Notebook for predicting Pitcher WAR from features via simple linear regression


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [2]:
# Load the by-year data (using retry files with fixed WAR merging)
data_yr = pd.read_csv("data/Pitchers_2015-2025_byYear_retry.csv")
print(f"Total rows: {len(data_yr)}")
print(f"Years available: {sorted(data_yr['year'].dropna().unique())}")
print(f"Players with 2025 data: {len(data_yr[data_yr['year'] == 2025]['mlbID'].unique())}")
print(f"Rows with WAR: {data_yr['WAR'].notna().sum()} / {len(data_yr)} ({data_yr['WAR'].notna().sum()/len(data_yr)*100:.1f}%)")
data_yr.head()


Total rows: 8968
Years available: [2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2023.0, 2024.0, 2025.0]
Players with 2025 data: 349
Rows with WAR: 8034 / 8968 (89.6%)


,Name,Age,#days,Lev,Tm,G,GS,W,L,SV,...,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff,Season,WAR
0,David Aardsma,33,3752,Maj-NL,Atlanta,33,0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,-0.13
1,Fernando Abad,29,3711,Maj-AL,Oakland,62,0,2.0,2.0,NaN,...,0.425,0.067,0.342,0.311,0.031,4.15,4.02,0.134,2015,0.18
2,A.J. Achter,26,3710,Maj-AL,Minnesota,11,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,-0.18
3,Austin Adams,28,3714,Maj-AL,Cleveland,28,0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,0.30
4,Nathan Adcock,27,3780,Maj-NL,Cincinnati,13,0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,-0.22


In [3]:
# Separate data into 2023-2024 (features) and 2025 (target)
data_2023_2024 = data_yr[data_yr['year'].isin([2023, 2024])].copy()
data_2025 = data_yr[data_yr['year'] == 2025].copy()

print(f"2023-2024 rows: {len(data_2023_2024)}")
print(f"2025 rows: {len(data_2025)}")
print(f"Unique players in 2023-2024: {data_2023_2024['mlbID'].nunique()}")
print(f"Unique players in 2025: {data_2025['mlbID'].nunique()}")

# Find players who have both 2023-2024 and 2025 data
players_with_2025 = set(data_2025['mlbID'].unique())
players_with_2023_2024 = set(data_2023_2024['mlbID'].unique())
common_players = players_with_2025.intersection(players_with_2023_2024)
print(f"Players with both 2023-2024 and 2025 data: {len(common_players)}")


2023-2024 rows: 738
2025 rows: 349
Unique players in 2023-2024: 512
Unique players in 2025: 349
Players with both 2023-2024 and 2025 data: 257


In [4]:
# Identify feature columns (exclude identifiers, year, and target)
exclude_cols = ['Name', 'mlbID', 'year', 'Season', 'WAR', 'Name_ev', 'Name_x', 
                'Lev', 'Tm', 'team', 'position', 'team_id']

# Get numeric columns that could be features
numeric_cols = data_2023_2024.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [col for col in numeric_cols if col not in exclude_cols and col != 'WAR']

print(f"Number of potential features: {len(feature_cols)}")
print(f"Sample features: {feature_cols[:10]}")


Number of potential features: 67
Sample features: ['Age', '#days', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R']


In [ ]:
# For each player, average their 2023-2024 features (handling NaN appropriately)
# We'll average numeric features, but need to be careful about which ones to average vs sum

def average_features_by_player(df, player_id_col='mlbID', feature_cols=None):
    """
    Average features across years for each player.
    Handles NaN by only averaging non-NaN values.
    """
    if feature_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        exclude_cols = ['year', 'Season', 'WAR']
        feature_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    # Group by player and average (NaN values are ignored in mean calculation)
    averaged = df.groupby(player_id_col)[feature_cols].mean().reset_index()
    
    # Also get player name for reference
    if 'Name' in df.columns:
        name_map = df.groupby(player_id_col)['Name'].first().reset_index()
        averaged = averaged.merge(name_map, on=player_id_col, how='left')
    
    return averaged

# Average 2023-2024 features for each player (for training)
features_avg_train = average_features_by_player(data_2023_2024, feature_cols=feature_cols)
print(f"Training features shape (2023-2024 averaged): {features_avg_train.shape}")

# Also average 2023-2024 features for players who have 2025 data (for prediction)
# This will be the same as features_avg_train, but we'll use it for consistency
features_avg = features_avg_train.copy()
print(f"Prediction features shape: {features_avg.shape}")
features_avg.head()


In [ ]:
# Prepare training data: 2023-2024 features -> 2023-2024 WAR per 162
# For each player in 2023-2024, calculate their WAR per 162
data_2023_2024_with_war = data_2023_2024.copy()
data_2023_2024_with_war['WAR_per_162'] = (data_2023_2024_with_war['WAR'] / data_2023_2024_with_war['G'].replace(0, pd.NA)) * 162

# Average WAR per 162 across 2023-2024 for each player (for training target)
war_per_162_train = data_2023_2024_with_war.groupby('mlbID')['WAR_per_162'].mean().reset_index()
war_per_162_train.columns = ['mlbID', 'WAR_per_162_train']

# Merge training features with training target
train_data = features_avg_train.merge(war_per_162_train, on='mlbID', how='inner')

print("Training Data (2023-2024):")
print(f"Shape: {train_data.shape}")
print(f"Players: {len(train_data)}")
print(f"\nWAR_per_162 (2023-2024 averaged) stats:")
print(train_data['WAR_per_162_train'].describe())
print(f"\nNaN in training target: {train_data['WAR_per_162_train'].isna().sum()}")

# Prepare test data: 2023-2024 features -> 2025 WAR per 162
# Ensure we have all required columns from data_2025
required_cols = ['mlbID', 'WAR', 'G', 'Name']
# Check which columns actually exist
available_cols = [col for col in required_cols if col in data_2025.columns]
target_2025 = data_2025[available_cols].copy()

# Calculate WAR per 162 games (handle division by zero)
if 'G' in target_2025.columns and 'WAR' in target_2025.columns:
    target_2025['WAR_per_162_2025'] = (target_2025['WAR'] / target_2025['G'].replace(0, pd.NA)) * 162
else:
    raise ValueError("Required columns 'G' or 'WAR' not found in data_2025")

# Merge features with 2025 target (for evaluation)
# Don't include 'G' from target_2025 since features_avg already has 'G' (from 2023-2024)
# We only need WAR_per_162_2025 and WAR for evaluation
merge_cols = ['mlbID', 'WAR_per_162_2025', 'WAR']
test_data = features_avg.merge(target_2025[merge_cols], on='mlbID', how='inner')

print("\nTest Data (2025):")
print(f"Shape: {test_data.shape}")
print(f"Players: {len(test_data)}")
print(f"\nWAR_per_162_2025 stats:")
print(test_data['WAR_per_162_2025'].describe())
print(f"\nNaN in test target: {test_data['WAR_per_162_2025'].isna().sum()}")

train_data.head()


In [ ]:
# Prepare training features and target (2023-2024)
train_data_clean = train_data.dropna(subset=['WAR_per_162_train']).copy()
# Only select feature columns that actually exist in the dataframe
available_train_features = [col for col in feature_cols if col in train_data_clean.columns]
X_train_full = train_data_clean[available_train_features].copy()
y_train_full = train_data_clean['WAR_per_162_train'].copy()

# Prepare test features and target (2025)
test_data_clean = test_data.dropna(subset=['WAR_per_162_2025']).copy()

# Debug: Check which feature columns are missing
missing_features = [col for col in feature_cols if col not in test_data_clean.columns]
if missing_features:
    print(f"Warning: {len(missing_features)} features missing from test_data: {missing_features[:10]}")

# Only select feature columns that actually exist in the dataframe
available_test_features = [col for col in feature_cols if col in test_data_clean.columns]
X_test = test_data_clean[available_test_features].copy()
y_test = test_data_clean['WAR_per_162_2025'].copy()

# Ensure both have the same features (use intersection)
common_features = list(set(available_train_features) & set(available_test_features))
X_train_full = X_train_full[common_features].copy()
X_test = X_test[common_features].copy()

print(f"Training set: {X_train_full.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Common features used: {len(common_features)}")
if len(common_features) < len(feature_cols):
    missing = set(feature_cols) - set(common_features)
    print(f"Features excluded (not in both datasets): {missing}")

# Handle NaN in features - we'll use median imputation for missing values
from sklearn.impute import SimpleImputer

# Fit imputer on training data
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train_full),
    columns=X_train_full.columns,
    index=X_train_full.index
)

# Transform test data with the same imputer
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"\nTraining NaN after imputation: {X_train_imputed.isna().sum().sum()}")
print(f"Test NaN after imputation: {X_test_imputed.isna().sum().sum()}")

# For consistency with variable names in later cells
X_train = X_train_imputed
y_train = y_train_full
X_test = X_test_imputed


In [ ]:
# Train simple linear regression on 2023-2024 data
# Model learns: 2023-2024 features -> 2023-2024 WAR per 162
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
# Training predictions (on 2023-2024 data)
y_train_pred = lr_model.predict(X_train)

# Test predictions (on 2025 data, using 2023-2024 features)
y_test_pred = lr_model.predict(X_test)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Model Performance:")
print(f"\nTraining (2023-2024 WAR per 162):")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE: {train_mae:.3f}")
print(f"  R²: {train_r2:.3f}")
print(f"\nTest (2025 WAR per 162):")
print(f"  RMSE: {test_rmse:.3f}")
print(f"  MAE: {test_mae:.3f}")
print(f"  R²: {test_r2:.3f}")


In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training set (2023-2024)
axes[0].scatter(y_train, y_train_pred, alpha=0.6)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual WAR per 162 (2023-2024)')
axes[0].set_ylabel('Predicted WAR per 162 (2023-2024)')
axes[0].set_title(f'Train Set: 2023-2024 (R² = {train_r2:.3f})')
axes[0].grid(True, alpha=0.3)

# Test set (2025)
axes[1].scatter(y_test, y_test_pred, alpha=0.6)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual WAR per 162 (2025)')
axes[1].set_ylabel('Predicted WAR per 162 (2025)')
axes[1].set_title(f'Test Set: 2025 (R² = {test_r2:.3f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Get feature importance (coefficients)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': lr_model.coef_
})
feature_importance['abs_coefficient'] = np.abs(feature_importance['coefficient'])
feature_importance = feature_importance.sort_values('abs_coefficient', ascending=False)

print("Top 20 Most Important Features (by absolute coefficient):")
print(feature_importance.head(20))


In [ ]:
# Make predictions for all players with 2023-2024 data (even if they don't have 2025 data)
# Model was trained on 2023-2024, now predicting 2025 WAR per 162

# Prepare all players' features (2023-2024 averaged)
# Use the same feature columns that were used for training
all_players_features = features_avg[common_features].copy()
all_players_features_imputed = pd.DataFrame(
    imputer.transform(all_players_features),
    columns=all_players_features.columns,
    index=all_players_features.index
)

# Make predictions (these are WAR per 162 for 2025)
all_predictions = lr_model.predict(all_players_features_imputed)

# Create results dataframe
predictions_df = features_avg[['mlbID', 'Name']].copy()
predictions_df['predicted_WAR_per_162_2025'] = all_predictions

# Recreate target_2025 to ensure it has all needed columns (in case cell 6 wasn't run)
target_2025_for_merge = data_2025[['mlbID', 'WAR', 'G', 'Name']].copy()
target_2025_for_merge['WAR_per_162_2025'] = (target_2025_for_merge['WAR'] / target_2025_for_merge['G'].replace(0, pd.NA)) * 162

# Merge with actual 2025 WAR per 162 if available
# Don't include 'G' in merge since predictions_df (from features_avg) already has 'G' from 2023-2024
# This avoids G_x/G_y column name conflict
merge_cols = ['mlbID', 'WAR_per_162_2025', 'WAR']
predictions_df = predictions_df.merge(
    target_2025_for_merge[merge_cols], 
    on='mlbID', 
    how='left'
)

# Add G_2025 separately if needed (rename to avoid conflict with G from features_avg)
if 'G' in target_2025_for_merge.columns:
    g_2025_data = target_2025_for_merge[['mlbID', 'G']].copy()
    g_2025_data = g_2025_data.rename(columns={'G': 'G_2025'})
    predictions_df = predictions_df.merge(g_2025_data, on='mlbID', how='left')

# Sort by predicted WAR per 162
predictions_df = predictions_df.sort_values('predicted_WAR_per_162_2025', ascending=False)

# Print results - include G from features_avg and G_2025 if available
print_cols = ['Name', 'predicted_WAR_per_162_2025', 'WAR_per_162_2025', 'WAR']
if 'G' in predictions_df.columns:
    print_cols.append('G')
if 'G_2025' in predictions_df.columns:
    print_cols.append('G_2025')

print("Top 20 Predicted WAR per 162 2025:")
print(predictions_df.head(20)[print_cols].to_string(index=False))

print(f"\nTotal players with predictions: {len(predictions_df)}")
print(f"Players with actual 2025 WAR per 162: {predictions_df['WAR_per_162_2025'].notna().sum()}")


In [ ]:
# Compare predictions vs actual for players with 2025 data
# This is the same as the test set evaluation, but showing individual player errors
comparison = predictions_df[predictions_df['WAR_per_162_2025'].notna()].copy()
comparison['error'] = comparison['predicted_WAR_per_162_2025'] - comparison['WAR_per_162_2025']
comparison['abs_error'] = np.abs(comparison['error'])

print("Players with largest prediction errors (2025):")
print(comparison.nlargest(10, 'abs_error')[['Name', 'predicted_WAR_per_162_2025', 'WAR_per_162_2025', 'error']].to_string(index=False))

print(f"\nMean Absolute Error (all players with 2025 data): {comparison['abs_error'].mean():.3f}")
print(f"RMSE (all players with 2025 data): {np.sqrt((comparison['error']**2).mean()):.3f}")
print(f"\nNote: Model trained on 2023-2024 WAR per 162, evaluated on 2025 WAR per 162")
